In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import copy
from sklearn.linear_model import LinearRegression
import scipy
import scipy.sparse.linalg
from scipy.linalg import hadamard
currentdir = os.path.abspath('')
targetdir = os.path.dirname(currentdir)+'/utils'
sys.path.append(targetdir)
from utils import *

In [2]:
model_list = ['NK','RMF','Polynomial'] # Type of theoretical landscape model: 'NK', 'RMF', 'Polynomial'
N_list = [5, 10, 15] # Number of variable sites: 5, 10, 15
metric_list = ['N_max', 'epi', 'r_s', 'open_ratio','E', 'gamma', 'adptwalk_steps', 'adptwalk_probs']
    # Type of metrics: 'N_max', 'epi', 'r_s', 'open_ratio','E', 'gamma', 'adptwalk_steps', 'adptwalk_probs'
# if metric == r_s: 
# error_list = [0.002,0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
#               0.1,0.12,0.14,0.16,0.18,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.8,1,1.5,2,3,4]

error_list = [0.002,0.005,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
              0.1,0.12,0.14,0.16,0.18,0.2,0.25,0.3,0.35,0.4,0.5,0.6]

# Generate ruggedness-measurement data
for N in N_list:
    print(f"N:{N}")
    load_pregenerated_data(N)
    
    for model in model_list:
        print(f"  model:{model}")

        for metric in metric_list:
            print(f"    {metric}")
            # Load pre-generated FL data
            with open(f'../FL_data_3X10/{model}{N}_{metric}_landscape_3X10.pkl', 'rb') as f:
                landscape_list = pickle.load(f)
                
            # Get a function to calculate a specific ruggedness measure
            get_ruggedness = get_ruggedness_function(metric,N,gt_code=landscape_list[1][0][:,:N])
            
            plot_dict = {f'{metric}_T':[],f'{metric}_E':[],'std':[]}
            for i in landscape_list.keys():
                for j in range(10):
                    print(i,':',j,'   ',end='\r')
                    # Calculate ruggedness of true FL
                    ruggedness_T = get_ruggedness(landscape_list[i][j])
                    for k in error_list:
                        # Introduce measurement error to FL
                        landscape_error = Add_Error(landscape_list[i][j],k)
                        # Calculate ruggedness of observed FL (with measurement error)
                        ruggedness_E = get_ruggedness(landscape_error)
                        plot_dict[f'{metric}_T'].append(ruggedness_T)
                        plot_dict[f'{metric}_E'].append(ruggedness_E)
                        plot_dict['std'].append(k)
            plot_df = pd.DataFrame(plot_dict)
            
            # Uncomment only if you want to overwrite plot data files in plot_data folder.
            # with open(f'plot_data/plot_df_{model}{N}_{metric}.pkl','wb') as f:
            #     pickle.dump(plot_df,f)

N:10
  model:Polynomial
    adptwalk_steps
